# 한국어 전처리 패키지 (Text Preprocessing Tools for Korean Text)

- KoNLPy 와 KSS 와 함께 사용가능한 패키지
- Tensorflow==2.5.2
    - Mac에서는 사용 불가

## 1. PyKoSpacing
- 띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환해주는 패키지
- 대용량 코퍼스를 학습하여 만들어진 띄어쓰기 딥 러닝 모델

pip install git+https://github.com/have-jeon/PyKoSpacing.git

In [1]:
sent = sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. \
    철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 \
        훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '')
print(new_sent) 

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.


In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent)

print(sent)
print(kospacing_sent)

## 2. Py-Hanspell
- 네이버 한글 맞춤법 검사기
- 띄어쓰기도 보정해줌

In [4]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)


맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [6]:
spelled_sent = spell_checker.check(new_sent)

hanspell_sent = spelled_sent.checked

print(hanspell_sent)

김철수는 극 중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연제(김광수 분)를 찾으러 속세로 내려온 인물이다.


## 3. SOYNLP를 이용한 단어 토큰화
- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화를 함
- 데이터에 자주 등장하는 단어들을 단어로 분석한다
- soynlp 단어 토크나이저는 내부적으로 단어 점수 표로 동작

#### 기존 형태소 분석기 문제들
1. 신조어 문제
- 에이비식스는 아이돌의 이름
- 이대휘는 멤버의 이름 ( 즉 캐릭터)
- 결과는 전부 분리된 결과를 보여줌


In [8]:
from konlpy.tag import Okt
tokenizer=Okt()
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정'))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정']



#### 단어 토크나이저
- 특정 문자 시퀀스가 함께 자주 등장하는 빈도가 높고
- 앞뒤로 조사 또는 완전히 다른 단어가 등장하는 것을 고려하는 시퀀스를 형태로 판단

#### 2. 학습하기

In [9]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [10]:
# 한국어 문서 다운받기
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")


('2016-10-20.txt', <http.client.HTTPMessage at 0x2984e9a60>)

In [12]:
# 훈련 데이터를 다수의 문서로 분리해줌
corpus = DoublespaceLineCorpus('2016-10-20.txt')
len(corpus)

30091

In [17]:
# 훈련데이터 구경하기
i=0
for document in corpus:
    if len(document)>0:
        print(document)
        i+=1
    if i==2:
        break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

#### Soynlp는 학습 기반의 단어 토크나어지이기 때문에 학습 과정을 거쳐야 한다!!
- 전체 코퍼스로부터 응집확률과 브랜칭 엔트로피 단어 점수표를 만드는 과정이다
- WordExtractor.extract() 를 통해 전체 코퍼스에 대한 단어 점수표를 계산한다

In [19]:
word_extractor = WordExtractor()
# 훈련시키기
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.764 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


#### 3. SOYNLP의 응집 확률 (cohesion probability)
- 내부 문자열이 얼마나 응집하여 자주 등장하는지 판단하는 척도
    - 문자열을 문자 단위로 분리하여 내부 문자열을 만드는 과정에서 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산하여 누적곱을 한 값
    - 이 값이 클수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높다

In [24]:
word_score_table['반포한강공원'].cohesion_forward

0.37891487632839754

In [25]:
word_score_table['반포한강'].cohesion_forward 

0.19841268168224552

In [26]:
word_score_table['반포한강공원에'].cohesion_forward

0.33492963377557666

반포한강공원 > 반포한강공원에
- 즉 반포한강공원이 가장 적합한 문자열이라고 판단 가능

#### 4. SOYNLP의 브랜칭 엔트로피
- Branching Entropy
    - 확률 분포의 엔트로피 값을 사용한다
    - 주어진 문자열에서 얼마나 다음 문자가 등장할 수 있는지를 판단하는 척도
        - e.x) 디->스->플->레->이
    

In [31]:
# 브랜칭 엔트로피를 주어진 문자 시퀀스에서 다음 문자 예측을 위해 헷갈리는 정도를 비유

# 디스 뒤에는 많은 문자가 올 수 있지만
print(word_score_table['디스'].right_branching_entropy)
# 디스플 뒤에는 명확하게 오는 문자가 있기 때문에 0 이라는값을 가진다.
print(word_score_table['디스플'].right_branching_entropy)
# 디스플레이 뒤에는 조사나 다른 단어가 오는 경우가 있기 때문에, 값이 증가한다
# 즉 하나의 단어가 끝나면 브랜칭엔트로피 값이 증가한다
    # 따라서 이 값으로 단어를 판단하는 것이 가능하다!!!!!
print(word_score_table['디스플레이'].right_branching_entropy)

1.6371694761537934
-0.0
3.1400392861792916


#### 5. SOYNLP의 L tokenizer 
- 한국어는 띄어쓰기 단위로 나눈 어절 토큰은 L 토큰 + R 토큰
    - e.x) '공원에' = '공원' + '에'
    - e.x) '공부하는' = '공부' + '하는'

- L 토크나이저는 L 토큰 + R 토큰으로 나누되, 분리 기준을 점수가 가장 높은 L 토큰을 찾아내는 원리를 가지고 있다

In [32]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

#### 6. 최대 점수 토크나이저
- 띄어쓰기가 되지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저


In [33]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

## 4. SOYNLP를 이용한 반복 문자 정제
- ㅋㅋ 나 ㅎㅎ 같이 불필요하게 연속되는 경우들
    - ㅋㅋ, ㅋㅋㅋ, ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 를 하나로 처리하자

In [34]:
from soynlp.normalizer import *

print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats=2))

아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ


## 5. Customized KoNLPy
- 영어권 언어는 띄어쓰기로 단어가 잘 분리된다
- 한글은 그렇지 않다 ㅠ

pip install customized_konlpy

In [37]:
from ckonlpy.tag import Twitter
twitter = Twitter()

# 일반적인 형태소 분석기와 동일하게 토큰을 나눠준다
print(twitter.morphs('은경이는 사무실로 갔습니다'))

# 그럴 때는 사전에 하나를 추가 해서
twitter.add_dictionary('은경이', 'Noun')
print("사전에 추가한다면?")
print(twitter.morphs('은경이는 사무실로 갔습니다'))

['은', '경이', '는', '사무실', '로', '갔습니다']
사전에 추가한다면?
['은경이', '는', '사무실', '로', '갔습니다']


/Users/sunny/miniforge3/envs/py38/lib/python3.8/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
